In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, Activation, BatchNormalization, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras import optimizers
import cv2
import numpy as np

In [ ]:
video_dir = '/content/drive/MyDrive/Multimodal Emotion Tracking/DEMO/Cropped Test Video'

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, Activation, BatchNormalization, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras import optimizers

class Cov3DModelClass:

    def define_model(self, NoOf3Dfilters, filtersize=(3,3,3), dense_neurons=512, dropout=0.25):
        model = Sequential()
        model.add(Conv3D(NoOf3Dfilters[0], filtersize, padding='same',
                 input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        for layer in range(len(NoOf3Dfilters)-1):
            model.add(Conv3D(NoOf3Dfilters[layer+1], filtersize, padding='same'))
            model.add(Activation('relu'))
            model.add(BatchNormalization())
            model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Flatten())
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        # Dense layer to output feature vector
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())

        optimiser = optimizers.Adam(lr=0.0002)
        model.compile(optimizer=optimiser, loss='mean_squared_error', metrics=['mean_squared_error'])

        return model

    def extract_features(self, video):
        model = self.define_model([32, 64, 128])  # Adjust filter sizes and number as needed
        features = model.predict(video)
        return features

def load_video(path, frames_to_sample, image_height, image_width):
    cap = cv2.VideoCapture(path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_interval = max(total_frames // frames_to_sample, 1)

    for i in range(frames_to_sample):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * sample_interval)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (image_width, image_height))
            frames.append(frame)
        else:
            break

    cap.release()
    frames = np.array(frames)
    frames = frames / 255.0  # Normalize to [0, 1]
    return frames

# Video directory path
video_directory_path = video_dir

# Load and preprocess the video
frames_to_sample = 30  # Number of frames to sample
image_height = 64
image_width = 64
channels = 3

# Create an instance of the model class
cov3d = Cov3DModelClass()
cov3d.frames_to_sample = frames_to_sample
cov3d.image_height = image_height
cov3d.image_width = image_width
cov3d.channels = channels

# Initialize an empty list to store features
all_features = []

# Process each video in the directory
for video_file in os.listdir(video_directory_path):
    video_path = os.path.join(video_directory_path, video_file)
    if not video_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        continue  # Skip non-video files

    # Load and preprocess the video
    video_frames = load_video(video_path, frames_to_sample, image_height, image_width)

    # Extract features
    features = cov3d.extract_features(np.expand_dims(video_frames, axis=0))

    # Append the features to the list
    all_features.append(features)

# Convert the list of features to a numpy array
all_features = np.concatenate(all_features, axis=0)
print(all_features)


1/1 [==============================] - 0s 323ms/step


1/1 [==============================] - 0s 314ms/step


1/1 [==============================] - 1s 526ms/step


1/1 [==============================] - 0s 429ms/step


1/1 [==============================] - 0s 314ms/step


1/1 [==============================] - 0s 325ms/step


1/1 [==============================] - 0s 366ms/step


1/1 [==============================] - 0s 321ms/step


1/1 [==============================] - 0s 333ms/step


1/1 [==============================] - 0s 337ms/step
[[0.00525217 0.004086   0.         ... 0.         0.         0.02732094]
 [0.07567284 0.01242337 0.05345793 ... 0.         0.02317854 0.04838737]
 [0.         0.         0.         ... 0.07870925 0.         0.        ]
 ...
 [0.10737726 0.00201567 0.         ... 0.         0.         0.        ]
 [0.01670427 0.00541949 0.00065322 ... 0.04721388 0.         0.        ]
 [0.04745089 0.         0.08129363 ... 0.         0.         0.        ]]


In [ ]:
len(all_features[0])

512

In [ ]:
print(all_features[0])

[0.00525217 0.004086   0.         0.         0.         0.
 0.         0.         0.06876712 0.01881582 0.         0.
 0.01011466 0.00637345 0.         0.03995518 0.         0.
 0.07165907 0.05512283 0.00744957 0.01882856 0.03331876 0.01671855
 0.         0.03845397 0.         0.07679415 0.01633142 0.
 0.02191704 0.         0.00130009 0.         0.         0.
 0.02163076 0.         0.         0.         0.06067532 0.07523557
 0.01158576 0.         0.         0.         0.06952833 0.
 0.         0.06148725 0.         0.03257024 0.03097127 0.
 0.         0.02384413 0.         0.04175206 0.0645711  0.00621987
 0.         0.08527883 0.         0.08006029 0.         0.06716938
 0.06337073 0.         0.         0.         0.         0.00710717
 0.03927256 0.         0.         0.09622052 0.04905767 0.
 0.00877132 0.         0.00366448 0.00322532 0.05533671 0.
 0.03510439 0.00412373 0.04478259 0.09894927 0.         0.00392587
 0.08714505 0.         0.         0.         0.04247868 0.01230401
